In [1]:
!pip uninstall numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2463299 sha256=4c224468adbd1b816803da8e891d4895a852dfb2c0d218d8120cb553061ed2d6
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import pandas as pd
from collections import defaultdict
from surprise import Reader, Dataset, SVD, dump, NMF, accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
# from sklearn.model_selection import train_test_split

In [2]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [11]:
# review_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/review_df.csv')
# movie_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/movie_df.csv')
ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_df.csv', usecols=['userId', 'movieId', 'rating'])
# ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/train.csv', usecols=['userId', 'movieId', 'rating'])

In [4]:
# ratings = review_df[['authorId', 'movieId', 'type']]
# ratings.loc[:,'type'] = ratings['type'].map({'Позитивный':5, 'Нейтральный':3, 'Негативный':1})
# ratings = ratings[['authorId', 'movieId', 'type']]
# ratings.rename(columns={'type':'rating'}, inplace=True)
ratings

,userId,movieId,rating
0,1.0,342.0,5.0
1,4.0,342.0,4.0
2,5.0,342.0,4.0
3,7.0,342.0,4.0
4,8.0,342.0,5.0
...,...,...,...
10291614,168183.0,28279.0,3.0
10291615,818517.0,28452.0,1.0
10291616,1696664.0,28452.0,3.0
10291617,172902.0,28452.0,3.0


In [5]:
reader = Reader(rating_scale=(1,5))

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, train_size=0.1, stratify=None)
train.to_csv('/content/drive/MyDrive/Kinopoisk/rating_train_df.csv')
del test
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
param_grid = {"n_epochs": [5, 10, 20], "lr_all": [0.002, 0.005], "reg_all": [0.002, 0.005], "n_factors":[50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae", "fcp"], cv=10, n_jobs=5, joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   58.5s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:  7.5min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 20.0min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed: 45.5min
[Parallel(n_jobs=5)]: Done 360 out of 360 | elapsed: 69.0min finished


In [19]:
# # best FCP score
print(f'Best FCP score is {gs.best_score["fcp"]}')
# print(gs.best_score["fcp"])

print('Combination of parameters that gave the best FCP score:')
print(gs.best_params["fcp"])

NameError: name 'gs' is not defined

In [13]:
# # best RMSE score
# print(gs.best_score["mae"])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params["mae"])

0.7234985701408936
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.005, 'n_factors': 50}


In [14]:
# # best RMSE score
# print(gs.best_score["rmse"])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params["rmse"])

0.9372344873893317
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.005, 'n_factors': 50}


In [10]:
reader = Reader(rating_scale=(1,5))

In [6]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
del ratings
trainset, testset = train_test_split(data, test_size=0.2)

In [7]:
svd=SVD(n_epochs=10, lr_all=0.005, reg_all=0.005, n_factors=50)
svd.fit(trainset)
predictions = svd.test(testset)

In [22]:
print(f"SVD model's RMSE {accuracy.rmse(predictions)}")
print(f"SVD model's MAE {accuracy.mae(predictions)}")
print(f"SVD model's FCP {accuracy.fcp(predictions)}")

RMSE: 0.8534
SVD model's RMSE 0.8533956848583474
MAE:  0.6499
SVD model's MAE 0.649879665470957
FCP:  0.6852
SVD model's FCP 0.6852209700369176


In [9]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

# Precision and recall can then be averaged over all users
print(f"SVD's model's precision at 10 {sum(prec for prec in precisions.values()) / len(precisions)}")
print(f"SVD' model's recall at 10 {sum(rec for rec in recalls.values()) / len(recalls)}")

SVD's model's precision at 10 0.7639212638275191
SVD' model's recall at 10 0.6582731669283072


In [25]:
train=pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_train_df.csv')

In [27]:
# param_grid = {"n_epochs": [25, 50, 75], "biased": [True, False], "n_factors":[15, 30, 60]}
# data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
# gs = GridSearchCV(NMF, param_grid, measures=["rmse",'mae', "fcp"], cv=10, n_jobs=10, joblib_verbose=5)
# gs.fit(data)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed: 23.6min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed: 101.2min
[Parallel(n_jobs=10)]: Done 180 out of 180 | elapsed: 135.1min finished


In [28]:
# # best FCP score
# print(f'Best FCP score is {gs.best_score["fcp"]}')
# # print(gs.best_score["fcp"])

# print('Combination of parameters that gave the best FCP score:')
# print(gs.best_params["fcp"])

Best FCP score is 0.5519708415108647
Combination of parameters that gave the best FCP score:
{'n_epochs': 25, 'biased': False, 'n_factors': 60}


In [32]:
# # # best RMSE score
# print(f'Best RMSE score is {gs.best_score["rmse"]}')
# # print(gs.best_score["rmse"])

# print('Combination of parameters that gave the best RMSE score:')
# print(gs.best_params["rmse"])

Best RMSE score is 0.9938639052245837
Combination of parameters that gave the best RMSE score:
{'n_epochs': 25, 'biased': True, 'n_factors': 15}


In [31]:
# # best MAE score
# print(f'Best MAE score is {gs.best_score["mae"]}')
# # print(gs.best_score["mae"])

# print('Combination of parameters that gave the best MAE score:')
# print(gs.best_params["mae"])

Best MAE score is 0.7619123515118941
Combination of parameters that gave the best MAE score:
{'n_epochs': 25, 'biased': True, 'n_factors': 15}


In [12]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
del ratings
trainset, testset = train_test_split(data, test_size=0.2)

In [13]:
nmf=NMF(n_epochs = 25, biased = True, n_factors = 15)
nmf.fit(trainset)

In [14]:
predictions = nmf.test(testset)

In [8]:
print(f"NMF model's RMSE {accuracy.rmse(predictions)}")
print(f"NMF model's MAE {accuracy.mae(predictions)}")
print(f"NMF model's FCP {accuracy.fcp(predictions)}")

RMSE: 1.2144
NMF model's RMSE 1.21444197464549
MAE:  0.9124
NMF model's MAE 0.9123753406237207
FCP:  0.5492
NMF model's FCP 0.5492122073840009


In [15]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

# Precision and recall can then be averaged over all users
print(f"NMF model's precision at 10 {sum(prec for prec in precisions.values()) / len(precisions)}")
print(f"NMF model's recall at 10 {sum(rec for rec in recalls.values()) / len(recalls)}")

NMF model's precision at 10 0.7168170701597053
NMF model's recall at 10 0.6113462019861439


In [11]:
trainset = data.build_full_trainset()
svd=SVD(n_epochs=20, lr_all=0.002, reg_all=0.005, n_factors=50)
svd.fit(trainset)


In [12]:
dump.dump('/content/drive/MyDrive/Kinopoisk/svd_dump_file1', algo=svd)